# Source detection

In [1]:
import subprocess
# import multiprocess
import numpy as np
from astropy.io import fits
from astropy import units as u
import matplotlib.pyplot as plt
from astropy import wcs
import os
from tqdm import tqdm
from astropy.coordinates import SkyCoord
import warnings
warnings.filterwarnings("ignore")

In [2]:
plt.style.use("default")
plt.rc('xtick', direction='in', top=True)
plt.rc('ytick', direction='in', right=True)
plt.rc('axes', linewidth=1.15)

plt.rc("mathtext", fontset="dejavuserif")

In [3]:
if not os.path.exists('../Data/Source_cat'):
    os.system('mkdir ../Data/Source_cat')

if not os.path.exists('../Data/Source_cat/BG_maps'):
    os.system('mkdir ../Data/Source_cat/BG_maps')

image_file = "../Data/Images/merged_image_200_2300.fits"
expmap_file = "../Data/Images/merged_expmap_200_2300.fits"

In [4]:
LOG_file = open("../Data/Source_cat/process.log", "w+")

## ERMASK:
It generates a detection mask for the eSASS source detection chain. The resulting product is a FITS image with values 0 and 1, where 1 indicates the image area on which the subsequent source detection tasks will be executed.

In [5]:
def run_ermask(exposure_map, output_mask_file, log_file=None):
    subprocess.run(["ermask", 
                    f"expimage={exposure_map}", 
                    f"detmask={output_mask_file}",
                    ], stdout=log_file, stderr=log_file)

In [6]:
output_mask_file = "../Data/Source_cat/detmask.fits"

if os.path.exists(output_mask_file):
    os.remove(output_mask_file)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_ermask(expmap_file, output_mask_file, log_file=log_file)

## ERBOX (local):
This step performs sliding box source detection in local mode. The objective of the local detection step is to create an initial list of source positions for the ERBACKMAP task (described below), which then generates a background map from a source-free image.

In [7]:
def run_erbox(image_file, exposure_map, detmask_file, output_boxlist, bkg_map=None, bg_image_flag="N", ecf=1, emin=200, emax=2300, log_file=None):
    if bg_image_flag=="N":
        subprocess.run(["erbox", 
                        f"images={image_file}", 
                        f"expimages={exposure_map}",
                        f"detmasks={detmask_file}",
                        f"boxlist={output_boxlist}",
                        f"emin={emin}",
                        f"emax={emax}",
                        f"bkgima_flag={bg_image_flag}",
                        f"ecf={ecf}",
                        ], stdout=log_file, stderr=log_file)
    else:
        subprocess.run(["erbox", 
                        f"images={image_file}", 
                        f"expimages={exposure_map}",
                        f"detmasks={detmask_file}",
                        f"boxlist={output_boxlist}",
                        f"emin={emin}",
                        f"emax={emax}",
                        f"bkgimages={bkg_map}",
                        f"ecf={ecf}",
                        ], stdout=log_file, stderr=log_file)

In [8]:
output_boxlist_local = "../Data/Source_cat/boxlist_local.fits"

if os.path.exists(output_boxlist_local):
    os.remove(output_boxlist_local)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_erbox(image_file, expmap_file, output_mask_file, 
              output_boxlist_local, log_file=log_file)

## ERBACKMAP

In [9]:
def run_erbackmap(image_file, exposure_map, detmask_file, boxlist_file, output_bkgmap, output_cheesemask, emin=200, emax=2300, log_file=None):
    subprocess.run(["erbackmap", 
                    f"image={image_file}", 
                    f"expimage={exposure_map}",
                    f"detmask={detmask_file}",
                    f"boxlist={boxlist_file}",
                    f"bkgimage={output_bkgmap}",
                    f"cheesemask={output_cheesemask}",
                    # f"emin={emin}",
                    f"emax={emax}",
                    "cheesemask_flag=Y",
                    "clobber=Y",
                    ], stdout=log_file, stderr=log_file)

In [10]:
output_bkgmap = "../Data/Source_cat/BG_maps/bkg_map.fits"
output_cheesemask = "../Data/Source_cat/BG_maps/cheesemask.fits"

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_erbackmap(image_file, expmap_file, output_mask_file, 
                  output_boxlist_local, output_bkgmap, output_cheesemask, 
                  log_file=log_file)

## ERBOX (map):

In [11]:
output_boxlist_map = "../Data/Source_cat/boxlist_map.fits"

if os.path.exists(output_boxlist_map):
    os.remove(output_boxlist_map)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_erbox(image_file, expmap_file, output_mask_file, 
              output_boxlist_map, output_bkgmap, log_file=log_file)

## ERMLDET:

In [12]:
def run_ermldet(image_file, exposure_map, detmask_file, boxlist_file,
                bkg_map, output_mllist, output_sourceimage, emin=200, emax=2300, log_file=None):
    subprocess.run(["ermldet", 
                    f"mllist={output_mllist}", 
                    f"boxlist={boxlist_file}",
                    f"images={image_file}",
                    f"expimages={exposure_map}",
                    f"detmasks={detmask_file}",
                    f"bkgimages={bkg_map}",
                    f"srcimages={output_sourceimage}",
                    "extentmodel=gaussian",
                    f"emin={emin}",
                    f"emax={emax}",
                    "srcima_flag=Y"
                    ], stdout=log_file, stderr=log_file)

In [13]:
output_mllist = "../Data/Source_cat/mllist.fits"
output_sourceimage = "../Data/Source_cat/sourceimage.fits"

if os.path.exists(output_mllist):
    os.remove(output_mllist)

if os.path.exists(output_sourceimage):
    os.remove(output_sourceimage)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_ermldet(image_file, expmap_file, output_mask_file, 
                output_boxlist_map, output_bkgmap, 
                output_mllist, output_sourceimage, log_file=log_file)

## CATPREP:

In [15]:
def run_catprep(input_mllist, out_catfile, log_file=None):
    subprocess.run(["catprep", 
                    f"infile={input_mllist}", 
                    f"outfile={out_catfile}"
                    ], stdout=log_file, stderr=log_file)

output_catalog = "../Data/Source_cat/catalog.fits"

if os.path.exists(output_catalog):
    os.remove(output_catalog)

with open("../Data/Source_cat/process.log", "a") as log_file:
    run_catprep(output_mllist, output_catalog, log_file=log_file)

## Selecting point sources and extended sources:

In [14]:
cheesemask_file = 'cheesemask.fits'
pts_cat = "../../../eRASS1_Main.v1.1.fits"
detmask_file = "../Data/Source_cat/detmask.fits"

hdulist = fits.open(image_file)
ima = hdulist[0].data
prihdr = hdulist[0].header
pix2deg = prihdr['CDELT2'] # deg
xsize, ysize = ima.T.shape
# mask_0 = np.copy(ima) * 0 + 1.
mask_hdu = fits.open(detmask_file)
mask = mask_hdu[0].data

ima_wcs = wcs.WCS(prihdr, relax=False)
ima_racen, ima_deccen = prihdr['CRVAL1'], prihdr['CRVAL2']
ima_r = np.max((xsize,ysize)) / 2 * pix2deg # deg
ima_coord = SkyCoord(ima_racen * u.deg, ima_deccen * u.deg, frame = 'icrs')

cat_src = fits.open(pts_cat)[1].data
cat_src = cat_src[(cat_src.EXT == 0) & (cat_src.DET_LIKE_0 > 12)] # Select high S/N point sources with DET_LIKE>8
coord_src = SkyCoord(cat_src.RA * u.deg, cat_src.DEC * u.deg, frame = 'icrs')

# Convert RA and DEC of sources to pixel coordinates
pix_coords = ima_wcs.all_world2pix(np.column_stack((cat_src.RA, cat_src.DEC)), 0)
# Only consider pix_coords that are within the image pixel bounds
valid_pix_coords_mask = (pix_coords[:, 0] >= 0) & (pix_coords[:, 0] < (xsize-1)) & (pix_coords[:, 1] >= 0) & (pix_coords[:, 1] < (ysize-1))
pix_coords = pix_coords[valid_pix_coords_mask]

# Create a mask to keep only sources with mask value of 1
# Needs the previous step to remove pixel coordinates that do  not belong to image i.e. <0 or >image size
valid_sources_mask = mask[pix_coords[:, 1].astype(int), pix_coords[:, 0].astype(int)] == 1

# Apply the mask to cat_src
cat_src = cat_src[valid_pix_coords_mask][valid_sources_mask]

ra_src = cat_src.RA
dec_src = cat_src.DEC

# Fix the masking radius to 1arcmin. Needs to be modified for more accurate analysis.
ext_src = np.zeros(len(ra_src)) + 1/60 

def circle(X, Y):
    x, y = np.meshgrid(X, Y)
    rho = np.sqrt(x * x + y * y)
    return rho

x = np.arange(ysize)
y = np.arange(xsize)
for j in tqdm(range(len(ra_src))):
    pixim = ima_wcs.all_world2pix([[float(ra_src[j]), float(dec_src[j])]], 0)
    xp = pixim[0][0]
    yp = pixim[0][1]
    rho = circle(x - xp, y - yp) * pix2deg
    ii = np.where(rho <= ext_src[j])
    if len(ii) > 0:
        mask[ii] = 0

hdu = fits.PrimaryHDU(mask)
hdu.header.update(ima_wcs.to_header())
hdulist = fits.HDUList([hdu])
hdulist.writeto(cheesemask_file, overwrite=True)

In [13]:
with open('sources.reg', 'w') as f:
    for i in range(len(ra_src)):
        print(f"fk5; circle({ra_src[i]},{dec_src[i]},{ext_src[i]})", file=f)